In [2]:
import json
import nltk
with open('data/SciQ dataset/train.json', 'r') as rf:
    train = json.load(rf)

In [3]:
train[0]

{'correct_answer': 'mesophilic organisms',
 'distractor1': 'protozoa',
 'distractor2': 'gymnosperms',
 'distractor3': 'viruses',
 'question': 'What type of organism is commonly used in preparation of foods such as cheese and yogurt?',
 'support': 'Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.'}

In [4]:
def tokenize(sequence):
    tokens = [token.replace("``", '"').replace("''", '"') for token in nltk.word_tokenize(sequence)]
    return map(lambda x:x.encode('utf8'), tokens)

In [5]:
for index in range (len(train)):
    supportTokens = tokenize(train[index]['support'])
    questionTokens = tokenize(train[index]['question'])
    distractor1Tokens = tokenize(train[index]['distractor1'])
    distractor2Tokens = tokenize(train[index]['distractor2'])
    distractor3Tokens = tokenize(train[index]['distractor3'])
    answerTokens = tokenize(train[index]['correct_answer'])

In [6]:
def read_dataset(dataset, tier, vocab):
    support_data = []
    question_data = []

    for index in range (len(train)):
        supportTokens = tokenize(train[index]['support'])
        questionTokens = tokenize(train[index]['question'])
#         distractor1Tokens = tokenize(train[index]['distractor1'])
#         distractor2Tokens = tokenize(train[index]['distractor2'])
#         distractor3Tokens = tokenize(train[index]['distractor3'])
#         answerTokens = tokenize(train[index]['correct_answer'])

        support_ids = [str(vocab.get(w, qa_data.UNK_ID)) for w in supportTokens]
        question_ids = [str(vocab.get(w, qa_data.UNK_ID)) for w in questionTokens]
        
        support_data.append(' '.join(support_ids))
        question_data.append(' '.join(question_ids))

    return support_data, question_data

In [7]:
def initialize_vocab(vocab_path):
    if tf.gfile.Exists(vocab_path):
        rev_vocab = []
        with tf.gfile.GFile(vocab_path, mode="rb") as f:
            rev_vocab.extend(f.readlines())
        rev_vocab = [line.strip('\n') for line in rev_vocab]
        vocab = dict([(x, y) for (y, x) in enumerate(rev_vocab)])
        return vocab, rev_vocab
    else:
        raise ValueError("Vocabulary file %s not found.", vocab_path)

In [14]:
## Glove stuff - Pass pretrained embeddings as input
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

NameError: name 'vocabulary_size' is not defined

In [12]:

import tensorflow as tf
def setup_word_embeddings(embeddings, question_ids, support_ids):
    '''
        Create an embedding matrix (initialised with pretrained glove vectors and updated only if self.config.train_embeddings is true)
        lookup into this matrix and apply dropout (which is 1 at test time and self.config.dropout at train time)
    '''
    with tf.variable_scope("vocab_embeddings"):
        _word_embeddings = tf.Variable(embeddings, name="_word_embeddings", dtype=tf.float32, trainable= self.config.train_embeddings)
        question_emb = tf.nn.embedding_lookup(_word_embeddings, question_ids, name = "question") # (-1, Q, D)
        passage_emb = tf.nn.embedding_lookup(_word_embeddings, passage_ids, name = "passage") # (-1, P, D)
        # Apply dropout
        self.question = tf.nn.dropout(question_emb, self.dropout)


In [13]:
## Pass tokens (question/answer/support) as input